In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.utils.labels as foul
import os
import uuid

In [ ]:
def export_dataset(dir, class_name, dataset):
    classes = dataset.default_classes
    mask_targets = {i: label for i, label in enumerate(classes)}

    # Render instance segmentations in `Segmentation` format
    foul.objects_to_segmentations(
        dataset,
        "ground_truth",
        "segmentation",
        mask_targets=mask_targets,
    )

    # Export images and detection XML
    dataset.export(
        data_path= os.path.join(dir, class_name, "images"),
        #labels_path= os.path.join(dir, class_name, "labels")
        dataset_type=fo.types.VOCDetectionDataset,
        label_field="ground_truth",
    )

    # Export segmentations
    dataset.export(
        labels_path= os.path.join(dir, class_name, "masks"),
        dataset_type=fo.types.ImageSegmentationDirectory,
        label_field="segmentation",
    )
    
def download_dataset(samples_count, class_name, is_train, dir, seed = 51):
    # We use "test" for training, because "test" has bigger data and takes less to download.
    dir = os.path.join(dir, "train" if is_train else "validation")
    dataset = foz.load_zoo_dataset(
        "open-images-v7",
        split= "test" if is_train else "validation",
        label_types=["segmentations"],
        classes = [class_name],
        max_samples= samples_count,
        seed=seed,
        shuffle=True,
        dataset_name=str(uuid.uuid4()),
        cleanup=True
        )
    
    export_dataset(dir, class_name, dataset)
    dataset.delete()
    

In [ ]:
dir_path = "/yourpath/"
download_dataset(samples_count=350, class_name="Cat", is_train=True, dir=dir_path)
download_dataset(samples_count=100, class_name="Cat", is_train=False, dir=dir_path)
download_dataset(samples_count=350, class_name="Dog", is_train=True, dir=dir_path)
download_dataset(samples_count=100, class_name="Dog", is_train=False, dir=dir_path)
download_dataset(samples_count=350, class_name="Bird", is_train=True, dir=dir_path)
download_dataset(samples_count=100, class_name="Bird", is_train=False, dir=dir_path)

In [ ]:
# session = fo.launch_app(dataset_1.view())
# print(session)